En este notebook se realiza un análisis de los resultados obtenidos durante la inferencia.

In [30]:
import json
from typing import List, Dict

Las métricas que tendremos en cuenta son:
- Hit Rate
- Recall
- Mean Reciprocal Rank

Primero, definimos las funciones de las métricas que vamos a usar

In [56]:
def hit_rate_in_retrieval(preds: List[Dict], truths: List[Dict], k: int = 5) -> float:
    aciertos = 0
    total = len(truths)

    for pred, truth in zip(preds, truths):
        relevantes = {(truth["document"], idx) for idx in truth["chunk_index"]}
        retrieved_topk = {
            (c["document_name"], c["chunk_index"])
            for c in pred.get("retrieved_chunks", [])[:k]
        }
        if relevantes & retrieved_topk:
            aciertos += 1

    return aciertos / total if total > 0 else 0.0

def hit_rate_in_response(preds: List[Dict], truths: List[Dict], k: int = 5) -> float:
    aciertos = 0
    total = len(truths)

    for pred, truth in zip(preds, truths):
        relevantes = {(truth["document"], idx) for idx in truth["chunk_index"]}
        retrieved_topk = {
            (c["document_name"], c["chunk_index"])
            for c in pred.get("references", [])[:k]
        }
        if relevantes & retrieved_topk:
            aciertos += 1

    return aciertos / total if total > 0 else 0.0

def mrr(preds: List[Dict], truths: List[Dict], k: int =5) -> float:
    rr_sum = 0.0
    total = len(truths)

    for pred, truth in zip(preds, truths):
        relevantes = {(truth["document"], idx) for idx in truth["chunk_index"]}
        retrieved = pred.get("retrieved_chunks", [])[:k]

        rank = None
        for i, c in enumerate(retrieved, start=1):  # rank empieza en 1
            if (c["document_name"], c["chunk_index"]) in relevantes:
                rank = i
                break

        if rank is not None:
            rr_sum += 1.0 / rank
        # si no hay match, contribuye con 0

    return rr_sum / total if total > 0 else 0.0

def recall_at_k(
    preds: List[Dict],
    truths: List[Dict],
    k: int = 10,
    average: Literal["macro", "micro"] = "macro",
    return_per_query: bool = False,
) -> Tuple[float, List[float] | None]:
    
    per_query_recalls = []
    global_hits = 0
    global_relevants = 0

    for pred, truth in zip(preds, truths):
        relevantes = {(truth["document"], idx) for idx in truth.get("chunk_index", [])}
        # top-k recuperados (como conjunto para evitar contar duplicados)
        retrieved_topk = {
            (c["document_name"], c["chunk_index"])
            for c in pred.get("retrieved_chunks", [])[:k]
        }

        hits = len(relevantes & retrieved_topk)
        total_rel = len(relevantes)

        # recall por query (si no hay relevantes, definimos 0.0 para evitar NaN)
        rq = (hits / total_rel) if total_rel > 0 else 0.0
        per_query_recalls.append(rq)

        global_hits += hits
        global_relevants += total_rel

    if average == "macro":
        score = sum(per_query_recalls) / len(per_query_recalls) if per_query_recalls else 0.0
    elif average == "micro":
        score = (global_hits / global_relevants) if global_relevants > 0 else 0.0
    else:
        raise ValueError("average debe ser 'macro' o 'micro'.")

    return (score, per_query_recalls if return_per_query else None)

Cargamos los datos de evaluación y los resultados (tanto con el reranker como sin él)

In [48]:
data = []
with open("../eval.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

results = []
with open("../results.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        results.append(json.loads(line))

results_no_reranker = []
with open("../results_no_reranker.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        results_no_reranker.append(json.loads(line))

## Resultados con reranker

In [62]:
print(f"Retrieval Hit Rate: {hit_rate_in_retrieval(results, data)}")
print(f"Response Hit Rate: {hit_rate_in_response(results, data)}")
print(f"Retrieval Recall: {recall_at_k(results, data)[0]}")
print(f"MRR: {mrr(results, data)}")

Retrieval Hit Rate: 1.0
Response Hit Rate: 1.0
Retrieval Recall: 0.95
MRR: 0.75


## Resultados sin reranker

In [64]:
print(f"Retrieval Hit Rate: {hit_rate_in_retrieval(results_no_reranker, data)}")
print(f"Response Hit Rate: {hit_rate_in_response(results_no_reranker, data)}")
print(f"Retrieval Recall: {recall_at_k(results_no_reranker, data)[0]}")
print(f"MRR: {mrr(results_no_reranker, data)}")

Retrieval Hit Rate: 0.9
Response Hit Rate: 0.9
Retrieval Recall: 0.9
MRR: 0.7666666666666666


Como podemos ver, con el reranker se obtiene mejores métricas que con sin él. Aunque el MRR es ligeramente mayor sin reranker, lo cual nos hace ver que rankea los chunks relevantes más arriba, en el resto de métricas es mejor, y dado que no estamos tan interesados en rankear arriba como en rankear los chunks relevantes @k, podemos concluir que el reranker mejora el pipeline del RAG.

**NOTA**: para hacer este estudio de forma más rigurosa, además de utilizar un dataset mucho mayor, se debería haber comparado los resultados usando las mismas queries. Dada la naturleza no determinista de los LLMs, cada inferencia puede dar una query de búsqueda diferente. De todas formas, en ambos casos se ha probado que el pipeline RAG aquí implementado funciona correctamente.